# GOES Data from GEE

In [4]:
import ee
import geemap

# Initialize the Earth Engine module.
ee.Initialize()

Map = geemap.Map()

# Band aliases.
BLUE = 'CMI_C01'
RED = 'CMI_C02'
VEGGIE = 'CMI_C03'
GREEN = 'GREEN'

# Number of bands in the EE asset, 0-based.
NUM_BANDS = 33

# Skipping the interleaved DQF bands.
BLUE_BAND_INDEX = (1 - 1) * 2
RED_BAND_INDEX = (2 - 1) * 2
VEGGIE_BAND_INDEX = (3 - 1) * 2
GREEN_BAND_INDEX = NUM_BANDS - 1

# Visualization range for GOES RGB.
GOES_MIN = 0.0
GOES_MAX = 0.7  # Alternatively 1.0 or 1.3.
GAMMA = 1.3

goes_rgb_viz = {
    'bands': [RED, GREEN, BLUE],
    'min': GOES_MIN,
    'max': GOES_MAX,
    'gamma': GAMMA
}

def apply_scale_and_offset(image):
    bands = [None] * NUM_BANDS  # Initialize with None to ensure correct length.
    
    for i in range(1, 17):
        band_name = f'CMI_C{str(100 + i)[-2:]}'
        offset = ee.Number(image.get(f'{band_name}_offset'))
        scale = ee.Number(image.get(f'{band_name}_scale'))
        bands[(i - 1) * 2] = image.select(band_name).multiply(scale).add(offset)

        dqf_name = f'DQF_C{str(100 + i)[-2:]}'
        bands[(i - 1) * 2 + 1] = image.select(dqf_name)

    # Green = 0.45 * Red + 0.10 * NIR + 0.45 * Blue
    green1 = bands[RED_BAND_INDEX].multiply(0.45)
    green2 = bands[VEGGIE_BAND_INDEX].multiply(0.10)
    green3 = bands[BLUE_BAND_INDEX].multiply(0.45)
    green = green1.add(green2).add(green3)
    bands[GREEN_BAND_INDEX] = green.rename(GREEN)

    return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames()))

collection = 'NOAA/GOES/16/MCMIPC/'
image_name = '2020211203115800000'
asset_id = collection + image_name
image = apply_scale_and_offset(ee.Image(asset_id))

# Assuming 'Map' is already defined in your environment (like in a Jupyter notebook).
Map.setCenter(-75, 37, 5)
Map.addLayer(image, goes_rgb_viz)

In [5]:
Map

Map(center=[37, -75], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [23]:
start_date = '2018-07-26'
end_date =  '2018-07-27'
date_of_interest = ee.Date('2018-07-26')

roi = ee.Geometry.Rectangle([-122.8, 40, -121, 41])

goes = ee.ImageCollection("NOAA/GOES/16/MCMIPC").filterDate(start_date, end_date).filterBounds(roi)

goes_sort = goes.map(lambda image: image.set(
        'dateDist',
        ee.Number(image.get('system:time_start')).subtract(date_of_interest.millis()).abs()))

# sort in ascending order by dateDist (so top image will correspond to date of interest)
goes_sorted = goes_sort.sort('dateDist')

# grab the first image from the sorted image collection
goes_img = goes_sorted.first()

# clip the image to the roi
goes_img_clipped = goes_img.clip(roi)

image = apply_scale_and_offset(ee.Image(goes_img_clipped))

Map.addLayer(image, goes_rgb_viz, 'clipped')

In [19]:
import xarray
ds = xarray.open_dataset(goes_sorted, engine='ee', crs='EPSG:4326', scale=0.25)

In [20]:
print(ds)

<xarray.Dataset> Size: 36GB
Dimensions:  (time: 272, lon: 1440, lat: 720)
Coordinates:
  * time     (time) datetime64[ns] 2kB 2018-07-26T00:02:19.700000 ... 2018-07...
  * lon      (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * lat      (lat) float64 6kB -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
Data variables: (12/32)
    CMI_C01  (time, lon, lat) float32 1GB ...
    DQF_C01  (time, lon, lat) float32 1GB ...
    CMI_C02  (time, lon, lat) float32 1GB ...
    DQF_C02  (time, lon, lat) float32 1GB ...
    CMI_C03  (time, lon, lat) float32 1GB ...
    DQF_C03  (time, lon, lat) float32 1GB ...
    ...       ...
    CMI_C14  (time, lon, lat) float32 1GB ...
    DQF_C14  (time, lon, lat) float32 1GB ...
    CMI_C15  (time, lon, lat) float32 1GB ...
    DQF_C15  (time, lon, lat) float32 1GB ...
    CMI_C16  (time, lon, lat) float32 1GB ...
    DQF_C16  (time, lon, lat) float32 1GB ...
Attributes: (12/18)
    date_range:             [1499644800000, 1647993600000]
   